In [1]:
import tensorflow as tf
import numpy as np
import os

/home/zhangbaojun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= "-1"
tf.reset_default_graph()

In [3]:
def read_TFRecords(example):
    feats=tf.parse_single_example(example,features={'features':tf.FixedLenFeature([],tf.string),'label':tf.FixedLenFeature([],tf.float32)})
    feature=tf.decode_raw(feats['features'],tf.float32)
    feature=tf.reshape(feature,[12,1])
    label=tf.reshape(feats['label'],[-1])
    return feature,label

In [4]:
test_name='/home/zhangbaojun/TrafficFlowPrediction-tensorflow/data/test_tfrecords'
test_data=tf.data.TFRecordDataset(test_name).map(read_TFRecords).shuffle(1000).batch(256)

In [5]:
iter=tf.data.Iterator.from_structure(test_data.output_types,test_data.output_shapes)
test_init_op=iter.make_initializer(test_data)
features,labels=iter.get_next()
print(np.shape(features),np.shape(labels))

(?, 12, 1) (?, 1)


In [6]:
#x=tf.placeholder(dtype=tf.float32,shape=[None,12,1],name="features")
#y=tf.placeholder(dtype=tf.float32,shape=[None,1],name="labels")

In [7]:
# def load_model():
#     saver=tf.train.import_meta_graph('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/.meta')
#     saver.restore(sess,tf.train.latest_checkpoint('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/'))
#     #saver=tf.train.get_checkpoint_state("/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/.meta")
#     #saver.restore(sess,ckpt.model_checkpoint_path)
#     return saver

In [42]:
def performance():
    Y_test=sess.run(Y,feed_dict={Y:sess.run(labels)})
    expectation = tf.cast(Y_test, tf.float32)
    pre=tf.get_collection("pred_network")[0]
    predictions=sess.run(pre,feed_dict={X:sess.run(features)})
    prediction=tf.convert_to_tensor(predictions)
    mae_metric,mae_update=tf.metrics.mean_absolute_error(expectation,prediction)
    mse_metric,mse_update=tf.metrics.mean_squared_error(expectation,prediction)
    
    residual = tf.reduce_sum(tf.square(tf.subtract(expectation, prediction)))
    total = tf.reduce_sum(tf.square(tf.subtract(expectation, tf.reduce_mean(expectation))))
    r2 = tf.subtract(1.0, tf.div(residual, total))
    
    tf.local_variables_initializer().run()
    print("MAE:",sess.run(mae_update),"MSE:",sess.run(mse_update),"R_squared:",sess.run(r2))
    
    


In [43]:
#with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:

In [44]:
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
init=tf.global_variables_initializer()
local=tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    sess.run(test_init_op)
    saver=tf.train.import_meta_graph('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/my_model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/'))
    graph=tf.get_default_graph()
    X=graph.get_operation_by_name("X").outputs[0]
    Y=graph.get_operation_by_name("Y").outputs[0]
    for i in range(0,5):
        performance()
    

    

INFO:tensorflow:Restoring parameters from /home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/my_model
MAE: 0.17415032 MSE: 0.043218356 R_squared: -0.18943882
MAE: 0.19525176 MSE: 0.053437438 R_squared: -0.51535547
MAE: 0.17608178 MSE: 0.044521406 R_squared: -0.337865
MAE: 0.18212758 MSE: 0.04760856 R_squared: -0.3584845
MAE: 0.17218287 MSE: 0.044207226 R_squared: -0.31544673


In [ ]:
with tf.Session() as sess1:
    sess1.run(init)
    saver1=tf.train.import_meta_graph('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/my_model.meta')
    saver1.restore(sess1,tf.train.latest_checkpoint('/home/zhangbaojun/TrafficFlowPrediction-tensorflow/model/'))
    graph1=tf.get_default_graph()
    varible1_names=[v.name for v in tf.global_variables()]
    values1=sess1.run(varible1_names)
    for k,v in zip(varible1_names,values1):
        print("Variable:",k)
        print("v.shape:",v.shape)
        #print("v.values:",v)